# CLIP tutorial written by Diana

## Intallation

Install miniconda:
```
cd $HOME
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash ~/Miniconda3-latest-Linux-x86_64.sh
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r
# move it to the zfs will not allow to use conda on other nodes, but /home dir only has a limited amount of space
mv ~/miniconda3 /zfs/ai4good/student/<your_usernamels>/miniconda3
ln -s /zfs/ai4good/student/<your_usernamels>/miniconda3 ~/miniconda3
```
Make a virtual enviroment (pytorch needs an older python than 13)
```
conda create --name myenv python=3.11
conda activate myenv # to deactivate run conda deactivate




python -m venv pytorch_env
source pytorch_env/bin/activate


conda create -n py311 python=3.11.6
source ~/pytorch_env/bin/activate
pip install --upgrade pip



# source ~/.bashrc
# conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0 # will give an error that python is outdated
conda create -y -n torch171 python=3.13
conda activate torch171
# conda config --env --add channels pytorch
# conda config --env --add channels conda-forge
# conda config --env --set channel_priority strict

```
Install Pytorch:
```
conda install -y pytorch=1.7.1 torchvision=0.8.2 cudatoolkit=11.0 -c pytorch -c conda-forge

```

Create enviroment with newer Python:
```
conda create -y -n torch171 python=3.8
conda activate torch171

```

## code sample